# Opening Large Datafiles

Sometimes you might be required to explore datafiles that are too large to fit as a whole dataframe into the memory of your computer. To get around this problem, we can load just a small amount of the dataset to do some intial analysis, and then filter it to discard any information that we don't need. We can also use this approach to just import the dataset piece by piece, if the final dataframe is not too large to fit in memory, as the process of importing a large datafile into a DataFrame will roughly take twice as much memory as the final DataFrame.

### Loading a chunk of a datafile

In [1]:
import pandas as pd

The code in the cell below creates an `iterator`. This is an object that iterates along a series of results, returning just one every time it is called. This series can itself be an iterable object, like a list or tuple, but it can also be the result of a function that changes each time it is called. For more information on iterators and functions, look up the `yield` function in python. But for now, the important thing to know is that we can iterate over the iterator in a `For` loop, and it will return the next `chunksize` lines of the datafile each time. In this case, it will return the next 10 lines of data.

In [19]:
chunk_iterator = pd.read_csv("data/HoC-GE2019-results-by-candidate.csv", chunksize=3)

To demonstrate how this works, we'll import just one chunk, and display it.

In [24]:
temp_df = next(chunk_iterator)
temp_df.head()

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,party_name,party_abbreviation,firstname,surname,gender,sitting_mp,former_mp,votes,share,change
9,W07000058,W92000004,Aberconwy,Clwyd,Wales,Wales,County,Plaid Cymru,PC,Lisa,Goodier,Female,No,No,2704,0.084858,-0.013742
10,W07000058,W92000004,Aberconwy,Clwyd,Wales,Wales,County,Liberal Democrat,LD,Jason,Edwards,Male,No,No,1821,0.057147,0.027878
11,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,Scottish National Party,SNP,Kirsty,Blackman,Female,Yes,Yes,20205,0.540053,0.127342


And if we run the same code again, without resetting the iterator, we should get a different subset of the data.

In [25]:
temp_df = next(chunk_iterator)
temp_df.head()

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,party_name,party_abbreviation,firstname,surname,gender,sitting_mp,former_mp,votes,share,change
12,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,Conservative,Con,Ryan,Houghton,Male,No,No,7535,0.201401,-0.025522
13,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,Labour,Lab,Nurul Hoque,Ali,Male,No,No,4939,0.132013,-0.168093
14,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,Liberal Democrat,LD,Isobel,Davidson,Female,No,No,2846,0.076070,0.030011


Using this approach, we can import the datafile piece by piece without having to hold all of it in memory at the same time. 
However, there are some important things to note about iterators:

1) The iterator will iterate along the function until it reaches the end. That means that if your iterable or function from which you're generating the iterator (in this case, our `.csv` datafile) has an end, the iterator will eventually start returning nothing.

2) An iterator will remember where it was when you try to use it again. If you obtain any values from an iterator, and then attempt to loop through it again, it will not start at the beginning but will instead start wherever it left off. If you iterated all the way through to the end of it, and attempted to get another value from it, it will just continue to return None. To get around this, you should recreate an iterator each time you intend to use it so that you do not accidentally miss any data from it. We'll demonstrate that currently.

### Initial Data Exploration

To start with our exploration, we'll want to get a quick look at a small subset of the data. This can allow us to check that it is importing okay, and look for any data that we can filter out and discard to make our final DataFrame smaller, be that entire columns or rows. Let's just import one chunk and look at it for now, remembering to recreate the iterator so that we always get the same chunk each time we run this next cell.

In [37]:
chunk_iterator = pd.read_csv("data/HoC-GE2019-results-by-candidate.csv", chunksize=20) # Now with a chunksize of 20
temp_df = next(chunk_iterator)
temp_df.head()

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,party_name,party_abbreviation,firstname,surname,gender,sitting_mp,former_mp,votes,share,change
0,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Labour,Lab,Stephen,Kinnock,Male,Yes,Yes,17008,0.538262,-0.142933
1,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Conservative,Con,Charlotte,Lang,Female,No,No,6518,0.206279,0.028901
2,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Brexit Party,Brexit,Glenda,Davies,Female,No,No,3108,0.098361,NaN
3,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Plaid Cymru,PC,Nigel,Hunt,Male,No,No,2711,0.085797,0.002804
4,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Liberal Democrat,LD,Sheila,Kingston-Jones,Female,No,No,1072,0.033926,0.015921


Here we have imported just the first 20 rows of the datafile, and displayed the first 5. You should hopefully be able to see all of the columns, using the horizontal scrollbar at the bottom if necessary, but if you can't then we can treat this like any other DataFrame for further exploration. Either way, let's use the `DataFrame.info()` function to have a look at the information for each column.

In [35]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
ons_id                20 non-null object
ons_region_id         20 non-null object
constituency_name     20 non-null object
county_name           20 non-null object
region_name           20 non-null object
country_name          20 non-null object
constituency_type     20 non-null object
party_name            20 non-null object
party_abbreviation    20 non-null object
firstname             20 non-null object
surname               20 non-null object
gender                20 non-null object
sitting_mp            20 non-null object
former_mp             20 non-null object
votes                 20 non-null int64
share                 20 non-null float64
change                15 non-null float64
dtypes: float64(2), int64(1), object(14)
memory usage: 2.8+ KB


Now we've seen all of the column info, it can be useful to have a look at what values we have in each column, using the `Series.unique()` function. It's worth noting here that this will NOT be representative of the final DataFrame, as we haven't imported all of the data, but it can help us get an idea of what we can expect in each column. If you need to look at more rows of data to start with, you can either increase the value of the `chunksize` in the import function above, or import multiple chunks and concatenate them together (we'll get to that in a moment, as that's how we'll import the whole datafile).

In [38]:
for col in temp_df.columns:
    unique_values = temp_df[col].unique()
    print("COLUMN:", col)
    print("NUM_UNIQUE:", len(unique_values))
    print("VALUES:", unique_values, "\n")

COLUMN: ons_id
NUM_UNIQUE: 4
VALUES: ['W07000049' 'W07000058' 'S14000001' 'S14000002'] 

COLUMN: ons_region_id
NUM_UNIQUE: 2
VALUES: ['W92000004' 'S92000003'] 

COLUMN: constituency_name
NUM_UNIQUE: 4
VALUES: ['Aberavon' 'Aberconwy' 'Aberdeen North' 'Aberdeen South'] 

COLUMN: county_name
NUM_UNIQUE: 3
VALUES: ['West Glamorgan' 'Clwyd' 'Scotland'] 

COLUMN: region_name
NUM_UNIQUE: 2
VALUES: ['Wales' 'Scotland'] 

COLUMN: country_name
NUM_UNIQUE: 2
VALUES: ['Wales' 'Scotland'] 

COLUMN: constituency_type
NUM_UNIQUE: 2
VALUES: ['County' 'Borough'] 

COLUMN: party_name
NUM_UNIQUE: 8
VALUES: ['Labour' 'Conservative' 'Brexit Party' 'Plaid Cymru' 'Liberal Democrat'
 'Independent' 'Green' 'Scottish National Party'] 

COLUMN: party_abbreviation
NUM_UNIQUE: 8
VALUES: ['Lab' 'Con' 'Brexit' 'PC' 'LD' 'Ind' 'Green' 'SNP'] 

COLUMN: firstname
NUM_UNIQUE: 19
VALUES: ['Stephen' 'Charlotte' 'Glenda' 'Nigel' 'Sheila' 'Captain' 'Giorgia'
 'Robin' 'Emily' 'Lisa' 'Jason' 'Kirsty' 'Ryan' 'Nurul Hoque' 'Iso

We can now see that in this sample of 20 rows, we have results from 4 constituencies, but roughly 20 `firstnames` and `surnames`. This implies that we have a row for each of several candidates for each constituency, which matches with what we can see up in the sample dataframe above. We can also see that we have 2 columns (`sitting_mp`, `former_mp`) which appear to be filled with binary values (`yes` or `no`). Finally, we can see the last 3 columns appear to have numeric values, with `votes` being an integer value, and with `share` and `change` appearing to be percentages.

### Filtering Data

Now that we've had a look at our data, we can read in the data in its entirety, discarding any that we don't want. We're going to do 3 examples here:

1) Importing the whole datafile

2) Importing only the columns concerning the MP genders for each constituency

3) Importing only the rows of MPs belonging to the Conservative Party

#### Example 1 - Whole Datafile

In [56]:
df_full = pd.read_csv("data/HoC-GE2019-results-by-candidate.csv") # Datafile read in as a whole

df_1 = pd.DataFrame()
chunk_iterator = pd.read_csv("data/HoC-GE2019-results-by-candidate.csv", chunksize=1000)
for chunk in chunk_iterator:
    df_1 = pd.concat([df_1, chunk])
    
print("Iterated DataFrame is equal to normally imported DataFrame:", df_1.equals(df_full))

df_1.head()

Iterated DataFrame is equal to normally imported DataFrame: True


,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,party_name,party_abbreviation,firstname,surname,gender,sitting_mp,former_mp,votes,share,change
0,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Labour,Lab,Stephen,Kinnock,Male,Yes,Yes,17008,0.538262,-0.142933
1,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Conservative,Con,Charlotte,Lang,Female,No,No,6518,0.206279,0.028901
2,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Brexit Party,Brexit,Glenda,Davies,Female,No,No,3108,0.098361,NaN
3,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Plaid Cymru,PC,Nigel,Hunt,Male,No,No,2711,0.085797,0.002804
4,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Liberal Democrat,LD,Sheila,Kingston-Jones,Female,No,No,1072,0.033926,0.015921


#### Example 2 - Importing a subset of columns

In [53]:
df_2 = pd.DataFrame()
chunk_iterator = pd.read_csv("data/HoC-GE2019-results-by-candidate.csv", chunksize=1000)
desired_cols = ['ons_id','ons_region_id','constituency_name','county_name','region_name','country_name','constituency_type']
for chunk in chunk_iterator:
    filtered_chunk = chunk[desired_cols]
    df_2 = pd.concat([df_2, filtered_chunk])
df_2.tail()

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type
3315,E14001061,E12000003,York Central,North Yorkshire,Yorkshire and The Humber,England,Borough
3316,E14001062,E12000003,York Outer,North Yorkshire,Yorkshire and The Humber,England,County
3317,E14001062,E12000003,York Outer,North Yorkshire,Yorkshire and The Humber,England,County
3318,E14001062,E12000003,York Outer,North Yorkshire,Yorkshire and The Humber,England,County
3319,E14001062,E12000003,York Outer,North Yorkshire,Yorkshire and The Humber,England,County


#### Example 3 - Importing a subset of rows

In [54]:
df_3 = pd.DataFrame()
chunk_iterator = pd.read_csv("data/HoC-GE2019-results-by-candidate.csv", chunksize=1000)
for chunk in chunk_iterator:
    filtered_chunk = chunk.loc[ chunk['party_name'] == 'Conservative' ]
    df_3 = pd.concat([df_3, filtered_chunk])
df_3.head()

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,party_name,party_abbreviation,firstname,surname,gender,sitting_mp,former_mp,votes,share,change
1,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,Conservative,Con,Charlotte,Lang,Female,No,No,6518,0.206279,0.028901
7,W07000058,W92000004,Aberconwy,Clwyd,Wales,Wales,County,Conservative,Con,Robin,Millar,Male,No,No,14687,0.460913,0.014972
12,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,Conservative,Con,Ryan,Houghton,Male,No,No,7535,0.201401,-0.025522
18,S14000002,S92000003,Aberdeen South,Scotland,Scotland,Scotland,Borough,Conservative,Con,Douglas,Lumsden,Male,No,No,16398,0.359306,-0.062019
23,S14000003,S92000003,Airdrie and Shotts,Scotland,Scotland,Scotland,County,Conservative,Con,Lorraine,Nolan,Female,No,No,7011,0.176280,-0.055629
